# Process the JSON files into more usable formats

In [ ]:
# Load json file and convert it to a pandas dataframe

import json
import pandas as pd
import os
import random
import numpy as np




## Process JSON data into numpy picked files

In [ ]:
# load json files and convert them into numpy arrays

directory = 'uncombined-training-data/time-freq-1-mil/1-milisecond-positive-json-data/'
all_positive = []
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        with open(directory + filename) as f:
            data = json.load(f)
            # go through the data and convert it to a numpy array
            for row in data:
                row_list_with_label = []
                row_list_with_label.append(1)
                for col in row:
                    row_list_with_label.append([float(col["x"]), float(col["y"]), float(col["z"])])
            all_positive.append(row_list_with_label)                    

save_np_array = np.array(all_positive)

# save the numpy array to a file
np.save('positive.npy', save_np_array)

directory = 'uncombined-training-data/time-freq-1-mil/1-milisecond-negative-json-data/'
all_negative = []
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        with open(directory + filename) as f:
            data = json.load(f)
            # go through the data and convert it to a numpy array
            for row in data:
                row_list_with_label = []
                row_list_with_label.append(0)
                for col in row:
                    row_list_with_label.append([float(col["x"]), float(col["y"]), float(col["z"])])
            all_negative.append(row_list_with_label)   
            
np.save('negative.npy', np.array(all_negative))


            

## Process JSON data into pandas dataframes and saved as tsv files

In [ ]:
# Load all the json files at specific directory


directory = 'uncombined-training-data/time-freq-1-mil/1-milisecond-positive-json-data/'
allPositiveFiles = []
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        with open(directory + filename) as f:
            # df.add(json.load(f))
            allPositiveFiles.append(json.load(f))
    else:
        continue


allDataframes = []
for file in allPositiveFiles:
    tempDf = pd.DataFrame(file)
    for index, row in tempDf.iterrows():
        for index2, col in row.iteritems():
            if col != None:
                dictionaryToArray = []
                for i in col.values():
                    dictionaryToArray.append(float(i))
                tempDf.at[index, index2] = dictionaryToArray
            else:
                tempDf.at[index, index2] = [0,0,0]
    allDataframes.append(tempDf)

# combine all the dataframes into one
df = pd.concat(allDataframes, ignore_index=True)



# add one collumn dataframe at the begging of the dataframe



# find the row with the most non zero values
maxNonZero = 0
for index, row in df.iterrows():
    temp = 0
    for index2, col in row.iteritems():
        if index2 != 'Y':
            if col != [0,0,0]:
                temp += 1
    if temp > maxNonZero:
        maxNonZero = temp

# find the row with the most zero values
maxZero = 0
for index, row in df.iterrows():
    temp = 0
    for index2, col in row.iteritems():
        if index2 != 'Y':
            if col == [0,0,0]:
                temp += 1
    if temp > maxZero:
        maxZero = temp

minNonZero = maxNonZero - maxZero

print("what is the maxNonZero: ", maxNonZero)

df.reindex(range(maxNonZero), fill_value=None)
for index, row in df.iterrows():
    for index2, col in row.iteritems():
            if col != None and type(col) !=float:
                pass
            else:
                df.at[index, index2] = [0,0,0]

df.insert(0, 'Y', 1)

# load all the negative json files at specific directory
negativeDirectory = 'uncombined-training-data/time-freq-1-mil/1-milisecond-negative-json-data/'
allNegativeFiles = []
for filename in os.listdir(negativeDirectory):
    if filename.endswith(".json"):
        with open(negativeDirectory + filename) as f:
            allNegativeFiles.append(json.load(f))
    else:
        continue


allNegativeDataframes = []
for file in allNegativeFiles:
    tempDf = pd.DataFrame(file)
    for index, row in tempDf.iterrows():
        for index2, col in row.iteritems():
            if col != None:
                dictionaryToArray = []
                for i in col.values():
                    dictionaryToArray.append(float(i))
                tempDf.at[index, index2] = dictionaryToArray
            else:
                tempDf.at[index, index2] = [0,0,0]
    allNegativeDataframes.append(tempDf)

# combine all the dataframes into one
negativeDf = pd.concat(allNegativeDataframes, ignore_index=True)


temp = []
for index, row in negativeDf.iterrows():
    split_rows = []
    split_index = random.randint(minNonZero, maxNonZero)
    start_index = 0
    while split_index < len(row.values):
        temp_row = row.values[start_index:split_index]
        start_index = split_index
        split_index = start_index + random.randint(minNonZero, maxNonZero)

        allZero = True
        for i in temp_row:
            if i != [0,0,0]:
                allZero = False
                break
        doesnt_contain_list = True
        for i in temp_row:

            if type(i) == float:
                doesnt_contain_list = False  
        
        if allZero == False and doesnt_contain_list == True:
            temp.append(temp_row)

# convert the list to numpy array

    
expanded_negativeDf = pd.DataFrame(temp)
expanded_negativeDf.reindex(range(maxNonZero), fill_value=0)

for index, row in expanded_negativeDf.iterrows():
    for index2, col in row.iteritems():
            if col != None or col == 0 and type(col) !=float:
                pass
            else:
                expanded_negativeDf.at[index, index2] = [0,0,0]


expanded_negativeDf.insert(0, 'Y', 0)

expanded_negativeDf.to_csv('negative.tsv', sep='\t', index=False)



df = pd.concat([df, expanded_negativeDf], ignore_index=True)


# save the maxNonZero and maxZero to a file called data_info.txt
with open('data_info.txt', 'w') as f:
    f.write("Max non zero: " + str(maxNonZero) + "\n")
    f.write("Max zero: " + str(maxZero) + "\n")
    f.write("Min non zero: " + str(minNonZero) + "\n")
    f.write("Total rows: " + str(len(df)) + "\n")




# Save dataframe to tab separated file
df.to_csv('data.tsv', sep='\t', index=False)


